In [36]:
# ライブラリのインストール
%pip install transformers fugashi ipadic sentencepiece

from transformers import BertJapaneseTokenizer, BertModel
import torch
import numpy as np
import os
import pandas as pd


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # モデル出力の最初の要素がトークン埋め込みを含む
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        for batch_idx in range(0, len(sentences), batch_size):
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="max_length", max_length=512,
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        return torch.stack(all_embeddings)

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


# テキストファイルを読み込み、各文をリストに格納
def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = f.read().splitlines()
    return sentences


In [38]:

# モデルの初期化
model1 = SentenceBertJapanese("sonoisa/sentence-bert-base-ja-mean-tokens-v2")
model2 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese")
model3 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese-v3")
model4 = SentenceBertJapanese("tohoku-nlp/bert-base-japanese-char-v3")
model5 = SentenceBertJapanese("tohoku-nlp/bert-large-japanese-v2")
model6 = SentenceBertJapanese("tohoku-nlp/bert-large-japanese-char-v2")


c:\WorkSpace\Practice\env2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of

In [49]:
file_path = "E:\実験\テキスト/test/文章36.txt"
sentences = read_sentences_from_file(file_path)
embeddings1 = model1.encode(sentences)

In [50]:
cos_sim(embeddings1[0].numpy(), embeddings1[1].numpy())

0.6096285

In [41]:
for _,_, files in os.walk("E:\実験\テキスト/test"):
    print(files)
    print(len(files))

['文章22.txt', '文章23.txt', '文章24.txt', '文章25.txt', '文章01.txt', '文章02.txt', '文章04.txt', '文章06.txt', '文章07.txt', '文章09.txt', '文章10.txt', '文章11.txt', '文章14.txt', '文章15.txt', '文章17.txt', '文章18.txt', '文章19.txt', '文章21.txt', '文章27.txt', '文章29.txt', '文章33.txt', '文章36.txt', '文章42.txt', '文章45.txt', '文章46.txt', '文章26.txt']
26


In [43]:
for _,_, files in os.walk("E:\実験\テキスト/test"):
    print(files)
    for i in range(0, len(files)):
        file = str(files[i])
        num = file[-6:-4]
        # print(num)

        file_path = "E:\実験\テキスト/test/文章" + str(num) + ".txt"
        save_path = "C:\WorkSpace\Practice\コサイン類似度/文間/cossim/cossim_result" + str(num) + ".csv"
        sentences = read_sentences_from_file(file_path)

        # 文をエンコード
        embeddings1 = model1.encode(sentences)
        embeddings2 = model2.encode(sentences)
        embeddings3 = model3.encode(sentences)
        embeddings4 = model4.encode(sentences)
        embeddings5 = model5.encode(sentences)
        embeddings6 = model6.encode(sentences)

        cossim_result1 = []
        cossim_result2 = []
        cossim_result3 = []
        cossim_result4 = []
        cossim_result5 = []
        cossim_result6 = []

        for k in range(0, len(sentences)-1):
            cossim_result1.append(cos_sim(embeddings1[k].numpy(), embeddings1[k+1].numpy()))
            cossim_result2.append(cos_sim(embeddings2[k].numpy(), embeddings2[k+1].numpy()))
            cossim_result3.append(cos_sim(embeddings3[k].numpy(), embeddings3[k+1].numpy()))
            cossim_result4.append(cos_sim(embeddings4[k].numpy(), embeddings4[k+1].numpy()))
            cossim_result5.append(cos_sim(embeddings5[k].numpy(), embeddings5[k+1].numpy()))
            cossim_result6.append(cos_sim(embeddings6[k].numpy(), embeddings6[k+1].numpy()))


        df = pd.DataFrame({
            "sonoisa/sentence-bert-base-ja-mean-tokens-v2" : cossim_result1,
            "tohoku-nlp/bert-base-japanese" : cossim_result2,
            "tohoku-nlp/bert-base-japanese-v3" : cossim_result3,
            "tohoku-nlp/bert-base-japanese-char-v3" : cossim_result4,
            "tohoku-nlp/bert-large-japanese-v2" : cossim_result5,
            "tohoku-nlp/bert-large-japanese-char-v2" : cossim_result6
        })

        df.to_csv(save_path, mode="a", header=True, index=False)

        print(f"{num}を記録しました。")

['文章22.txt', '文章23.txt', '文章24.txt', '文章25.txt', '文章01.txt', '文章02.txt', '文章04.txt', '文章06.txt', '文章07.txt', '文章09.txt', '文章10.txt', '文章11.txt', '文章14.txt', '文章15.txt', '文章17.txt', '文章18.txt', '文章19.txt', '文章21.txt', '文章27.txt', '文章29.txt', '文章33.txt', '文章36.txt', '文章42.txt', '文章45.txt', '文章46.txt', '文章26.txt']


KeyboardInterrupt: 

In [ ]:


# テキストファイルから文を読み込む
file_path = "E:\実験\テキスト\実験テキスト_卒研B\使用済み文章\文章42.txt"  # 読み込むファイルのパス
save_path = "C:\WorkSpace\Practice\コサイン類似度\文間\cossim_result42.csv"
sentences = read_sentences_from_file(file_path)

# 文をエンコード
embeddings1 = model1.encode(sentences)
embeddings2 = model2.encode(sentences)
embeddings3 = model3.encode(sentences)
embeddings4 = model4.encode(sentences)
embeddings5 = model5.encode(sentences)
embeddings6 = model6.encode(sentences)


In [28]:
for i in range(1, len(sentences)):
    print(sentences[i])

一神教ではもっぱら一つの神だけが信仰の対象になるために、他の神々が排除され排他的な傾向が生まれる。
それに対して、多神教では多様な神々が信仰の対象となり、そこに優劣がつけられないために、異なる信仰に対しても寛容である。
そのような理解のもとに、一神教が批判の対象にされることが多い。
さらに、そこには「砂漠の宗教」と「森の宗教」の対立といった風土論が重ね合わされる。
一神教は自然環境が厳しい砂漠で生まれたために信仰に対して厳しく、豊かな自然に恵まれた森のなかで育まれた多神教とは性格が異なるというわけである。
こうした一神教と多神数との対比には、八百万の神を信仰する点でまさに多神数に分類される日本の宗教を、その寛容性において高く評価しようとする価値判断が強く働いている。
しかし、この対立、対比が果たして有効で適切なものなのかどうか、そもそも事実にもとづくものなのかどうかは再検討の余地がある。
砂漠の一神教の典型とされるのはイスラム教だが、それは砂漠のなかのオアシスにある都市に生まれた宗教であって、単純に砂漠の宗教とは言えず、むしろ「商売人の宗教」としてとらえるべきだという見解もある。
また、一神教のなかに多神教的な側面を見出すこともできる。
キリスト教の三位一体の教義においては父と子と聖霊が一体であるとされるが、その考え方に多神教的な神観を見出すことができる。
実際キリスト教では、時代を経るにつれて聖母マリアに対する信仰が高まりを見せ、父なる神やイエス・キリスト以上に信者たちの信仰を集めてきた。
私たちにはあまりなじみがないが、神学の世界においてもマリアの重要性は増してきている。
さらに、聖人、聖者の信仰もしだいに盛んになってきた。
信仰を守って殉教した人間が病気の治癒などの奇跡を引き起こすことで聖者（最初は福者）に列せられ、庶民の強い信仰を集めることになった。
この聖者に対する信仰は、日本の八百万の神に対する信仰にかなり似ている。
ご利益仰である点で共通しているからだ。
キリスト教以上に唯一絶対の神への信仰を強調するイスラム教においても、やはり聖者信仰が盛んである。
神の絶対性が過度に強調されると、人間界との関係は相当に希薄なものとなり、日常的な悩みや苦しみからの救いを神に期待できなくなる。
そこに、より身近な存在である聖者の役割があり、一神教にも多神教的な側面が見ら

In [29]:
cossim_result1 = []
cossim_result2 = []
cossim_result3 = []
cossim_result4 = []
cossim_result5 = []
cossim_result6 = []

for i in range(len(sentences)):
    cossim_result1.append(cos_sim(embeddings1[i].numpy(), embeddings1[i+1].numpy()))
    cossim_result2.append(cos_sim(embeddings2[i].numpy(), embeddings2[i+1].numpy()))
    cossim_result3.append(cos_sim(embeddings3[i].numpy(), embeddings3[i+1].numpy()))
    cossim_result4.append(cos_sim(embeddings4[i].numpy(), embeddings4[i+1].numpy()))
    cossim_result5.append(cos_sim(embeddings5[i].numpy(), embeddings5[i+1].numpy()))
    cossim_result6.append(cos_sim(embeddings6[i].numpy(), embeddings6[i+1].numpy()))

In [30]:
import pandas as pd
df = pd.DataFrame({
    "sonoisa/sentence-bert-base-ja-mean-tokens-v2" : cossim_result1,
    "tohoku-nlp/bert-base-japanese" : cossim_result2,
    "tohoku-nlp/bert-base-japanese-v3" : cossim_result3,
    "tohoku-nlp/bert-base-japanese-char-v3" : cossim_result4,
    "tohoku-nlp/bert-large-japanese-v2" : cossim_result5,
    "tohoku-nlp/bert-large-japanese-char-v2" : cossim_result6
})

df.to_csv(save_path, mode="a", header=True, index=False)